In [38]:
import numpy as np
from pymongo import MongoClient
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [63]:
class CnnScraper():
    
    def __init__(self):
        self.name = "CNN Web Scraper"
        self.page = 'https://www.cnn.com/politics'
        
    def initialize_database(self,website):
        client = MongoClient()
        self.cnn_db = client['breitbart']
        self.urls = self.cnn_db['urls']
        self.articles = self.cnn_db['articles']
        
    def get_urls(self):
        links=[]
        count=0
        pg = requests.get(self.page)
        if pg.status_code==200:
            soup = BeautifulSoup(pg.text,"lxml")
            data = soup.findAll('dic', {'class':"cd__status"})
        else:
            print("Failed to get page")

        for i in data:
            link= self.page +( i.find('a').get("href"))
            if link not in links:
                links.append(link)          
            print(f'Scraped {len(links)} urls.')
            time.sleep(2)
        #for link in links:
            #urls.insert_one({'link': link})
        return soup

In [64]:
c = CnnScraper()

In [65]:
c.initialize_database

<bound method CnnScraper.initialize_database of <__main__.CnnScraper object at 0x10e851e10>>

In [66]:
e = c.get_urls()

In [73]:
e.find_all('div', {'class':"media"})

[]

In [ ]:
def get_urls(x):
    links=[]
    count=0
    first_half = "https://www.breitbart.com"
    for num in x:
        pg = requests.get(f'https://www.breitbart.com/politics/page/{num}/')
        if pg.status_code==200:
            soup = BeautifulSoup(pg.text,"lxml")
            data = soup.findAll('div', {'class':"tC"})
        else:
            continue
            
        for i in data:
            link=first_half+(i.find('a').get("href"))
            if link not in links:
                links.append(link)          
        print(f'Scraped {len(links)} urls.')
        time.sleep(2)
    for link in links:
        urls.insert_one({'link': link})
    print("Done!!")


def get_articles(urls_of_articles):
    count=0
    for url in urls_of_articles:
        pg = requests.get(url)
        if pg.status_code==200:
            soup = BeautifulSoup(pg.text,"lxml")
            title = soup.find('h1').text
            author = soup.find('div', {"class": 'header_byline'}).find('a').text
            date = soup.find('div', {"class": 'header_byline'}).find('time').text
            shares = soup.find('span', {"class": 'acz5'}).text
            
            data = soup.findAll('div', {'class':"entry-content"})
            content=[]
            for i in data:
                for j in i.find_all('p'):
                    t = j.get_text()
                    content.append(t)
            content = " ".join(content)
            key = {"title":title}
            value = {'title':title, 'author':author, 'date': date, 'content': content}
            articles.update(key,value,upsert=True)
            count+=1
            print(f'Scraped {count} articles. ({title})')
            time.sleep(2)
        else:
            print('Failed to get page!')
            continue

websites = [urls.find()[i]['link'] for i in range(5000,7500)]
get_articles(websites)
print('Finished Scraping!')